# SOCATv2 FluxEngine Calculations

Here we calculate air-sea flux using adapted code from FluxEngine

## Reanalysis CO2 data to a constant temperature

The script function v2_f_confersion.py has been identified as the script that runs the fCO2 conversion
As we don't wish to grid our observations, and we have matched SST and WS data from satelites prior, we will feed our array directly into this funtion to reanalyse CO2 

In [44]:
#Standard + 3rd Party libs
import os
import numpy as np
import numpy.lib.recfunctions
import datetime
import argparse
import netCDF4
import glob
import multiprocessing
import pandas as pd;
os.chdir(os.path.join("C:\\Users\kabaldry\Documents\FluxEngine-master\\fluxengine_src\\tools\\reanalyse_socat"))
#Internal tool libs
import get_sst
import datenum
import v2_f_conversion
import netcdf_helper
import combine_nc_files
from pandas import DataFrame
os.chdir(os.path.join("C:\\Users\kabaldry\Documents\FluxEngine-master\SOCATv2\\tools"))
import required_f

### set up data frame for reanalysis using required packages

In [45]:
os.chdir(os.path.join("C:\\Users\kabaldry\Documents\FluxEngine-master\SOCATv2"))
SOCAT_file = 'SOCATv2_SO_sst_ice_31012020.tsv'

columnInfo = required_f.construct_column_info(year_col = 'yr', month_col = 'mon', day_col = 'day', hour_col = 'hh', minute_col = 'mm', second_col = 'ss', longitude_col = 'lon', latitude_col = 'lat', \
                                   salinity_col = 'sal', salinity_sub_col = 'WOA_SSS', SST_C_col = 'SST [deg.C]', Tequ_col = 'Tequ [deg.C]', air_pressure_col = 'PPPP [hPa]',  air_pressure_sub_col= 'NCEP_SLP [hPa]', air_pressure_equ_col= 'Pequ [hPa]', \
                                   fCO2_col= 'fCO2rec [uatm]', expocode_col = 'Expocode', socatversion = 6, notsocatformat = False);

data=required_f.ReadInData(inputfile=SOCAT_file,columnInfo=columnInfo, socatversion=6)

data_subset = data

data_subset=data_subset[np.where(np.isfinite(data_subset['fCO2']))]
#data_subset['longitude']=np.where(data_subset['longitude']>=180,
#                                            data_subset['longitude']-360,data_subset['longitude'])
#Finally we can remove columns which were only required for quality checks
#these are days,hours,mins,fCO2rec_flag
names=list(data_subset.dtype.names)
[names.remove(x) for x in ['fCO2_qc_flag']] #'day','mm','hh','ss',
names = [str(x) for x in names]; #Unicode strings can't be used to index, but are returned as column names. Weird.
data_subset=data_subset[names]

#Optionally update names to be more sensible - could do this at data import instead
#Note this ASSUMES the order of the column naming - FIXME: should do this more robustly #TMH: Fixed: Order is now determined by the columnInfo object after parsing command line args.
#newnames=['yr','mon','day','hh','mm','ss','lon','lat','sal','SST_C','Teq_C','P','Peq','sal_woa','P_ncep','fCO2_rec','expocode']
newnames=["expocode", "year", "month", "day", "hour", "minute", "second", "longitude", "latitude", "salinity", "sst", "T_equ", "air_pressure", "air_pressure_equ", "salinity_sub", "air_pressure_sub", "fCO2"];
data_subset.dtype.names=newnames


['expocode', 'year', 'month', 'day', 'hour', 'minute', 'second', 'longitude', 'latitude', 'salinity', 'sst', 'T_equ', 'air_pressure', 'air_pressure_equ', 'salinity_sub', 'air_pressure_sub', 'fCO2', 'fCO2_qc_flag']
Reading in data from SOCAT file: SOCATv2_SO_sst_ice_31012020.tsv
This can take a while for large datasets.



### prepare jds, Tcls and Peq_cls

In [46]:
os.chdir(os.path.join("C:\\Users\kabaldry\Documents\FluxEngine-master\SOCATv2"))
SOCAT_file2 = 'SOCATv2_SO_sst_ice_31012020.tsv'
df = pd.read_table(SOCAT_file2, sep="\t")
df_subset=df[np.isfinite(df['fCO2rec [uatm]'])]
#df_subset['lon']=np.where(df_subset['lon']>=180,
#                                            df_subset['lon']-360,df_subset['lon'])
jds = datenum.datenum_array(data_subset['year'], data_subset['month'], data_subset['day'], data_subset['hour'], data_subset['minute'], 0)
# no need to apply the subskin to skin estimate (note this is done in the v2_f_convert_f_SSH.py script in fluxengine) ast OISST is already subskin
Tcls = np.array(df_subset['sst'] + 273.15,'<f8')
#Tcls = Tcls.astype('<f8')
Peq_cls = data_subset['air_pressure_sub'] + 3.
#extrapolatetoyear = # we dont want to extrapolate to year... we are just interested in fluxes

C:\Users\kabaldry\AppData\Local\Continuum\anaconda3\envs\IOCCPEngineFlux\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead.
  This is separate from the ipykernel package so we can avoid doing imports until


### run reanalysis function 

In [47]:
conversion = v2_f_conversion.v2_f_conversion_wrap(jds,data_subset,Tcls,Peq_cls)

Writing ignored points to temp file: c:\users\kabaldry\appdata\local\temp\ignore_points_6wveua\ignored_points_8qwbyv


### append to original data frame

In [48]:
df_subset['SST_C'] =np.array(np.nan, '<f8')
df_subset['SST_C'][conversion['goodpoints']] = conversion['SST_C']

df_subset['Tcl_C'] =np.array(np.nan, '<f8')
df_subset['Tcl_C'][conversion['goodpoints']] = conversion['Tcl_C']

df_subset['fCO2_SST'] =np.array(np.nan, '<f8')
df_subset['fCO2_SST'][conversion['goodpoints']] = conversion['fCO2_SST']

df_subset['fCO2_Tym'] =np.array(np.nan, '<f8')
df_subset['fCO2_Tym'][conversion['goodpoints']] = conversion['fCO2_Tym']

df_subset['pCO2_SST'] =np.array(np.nan, '<f8')
df_subset['pCO2_SST'][conversion['goodpoints']] = conversion['pCO2_SST']

df_subset['pCO2_Tym'] =np.array(np.nan, '<f8')
df_subset['pCO2_Tym'][conversion['goodpoints']] = conversion['pCO2_Tym']

df_subset['qf'] =np.array(np.nan, '<f8')
df_subset['qf'][conversion['goodpoints']] = conversion['qf']


C:\Users\kabaldry\AppData\Local\Continuum\anaconda3\envs\IOCCPEngineFlux\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\kabaldry\AppData\Local\Continuum\anaconda3\envs\IOCCPEngineFlux\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\kabaldry\AppData\Local\Continuum\anaconda3\envs\IOCCPEngineFlux\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 

### Write reanalysis output to a file

In [49]:
 os.chdir(os.path.join("C:\\Users\kabaldry\Documents\FluxEngine-master\SOCATv2"))
 df_subset.to_csv('reanalysis_out_31012020.tsv',sep = '\t',index = False)

### FluxEngine output

In [50]:
#Set the working directory to the FluxEngine directory
import os

os.chdir(os.path.join("C:\\Users\kabaldry\Documents\FluxEngine-master"))
os.getcwd()
import argparse; #parsing command line arguments
import inspect; #stack
import fluxengine_src.fe_setup_tools_edit as setup;

In [51]:
returnCode, fe = setup.run_fluxengine(reanalysis_out = conversion,configFilePath = 'SOCATv2\socatv2_30012020.conf')

Executing on 'DOM-IMA-FLNMYM2' at 31/01/2020 18:09:23
Parsing settings file at: fluxengine_src\settings.xml
Converting pressure from Pa to mbar
Converting sstfnd from celsius to kelvin.
(ofluxghg_flux_calc, FluxEngine._run_fluxengine_) Using the saline skin model (0.100000 psu added to skin salinities)
(ofluxghg_flux_calc, FluxEngine._run_fluxengine_) Using SSTfnd data selection with correction for skin temperature (SSTskin = SSTfnd - 0)(ignoring SSTskin data in configuration file).
(ofluxghg_flux_calc, FluxEngine._run_fluxengine_) Using the RAPID model (from Woolf et al., 2016)
(rate_parameterisation.py: k_Wanninkhof2014.__call__) Using the Wanninkhof 2014 k parameterisation
(ofluxghg_flux_calc, FluxEngine._run_fluxengine_) SUCCESS writing file C:\Users\kabaldry\Documents\FluxEngine-master\output\socatv2\FE_output_31012020.tsv
Flux engine exited with exit code: 0
completed successfully.



In [52]:
os.chdir(os.path.join("C:\\Users\kabaldry\Documents\FluxEngine-master\output\socatv2"))
output_file = 'FE_output_31012020.tsv'
out = pd.read_table(output_file, sep="\t")
for x in out.columns:
    df_subset[x] = out[x]

C:\Users\kabaldry\AppData\Local\Continuum\anaconda3\envs\IOCCPEngineFlux\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [53]:
os.chdir(os.path.join("C:\\Users\kabaldry\Documents\FluxEngine-master\SOCATv2"))
df_subset.to_csv('FluxEngine_out_30012020.tsv',sep = '\t',index = False)

K calculation

In [ ]:
self.data["scskin"].fdata = schmidt_Wanninkhof2014(self.data["sstskinC"].fdata, nx, ny, runParams.GAS)

#based on Schmid relationship from Wanninkhof2014 - Relationship between wind speed and gas exchange over the ocean revisited, Limnology and Oceanography
def schmidt_Wanninkhof2014(sstC_fdata, nx, ny, gas):
    sc_fdata = array([missing_value] * nx*ny)

    if 'co2' in gas.lower():
        for i in arange(nx * ny):
            if (sstC_fdata[i] != missing_value):
                # relationship is only valid for temperatures <=30.0 oC
                sc_fdata[i] = 2116.8 + (-136.25 * sstC_fdata[i]) + (4.7353 * sstC_fdata[i]**2) + (-0.092307 * sstC_fdata[i]**3) + (0.0007555 * sstC_fdata[i]**4);
            else:
            # assigning invalid values
                sc_fdata[i] = missing_value
    return sc_fdata


class k_Wanninkhof2014(KCalculationBase):
    #def __init__(self, kb_weighting, kd_weighting):
    def __init__(self):
        self.name = self.__class__.__name__;
    
    def input_names(self):
        return ["windu10", "windu10_moment2", "scskin"];
    
    def output_names(self):
        return ["k"];
    
    def __call__(self, data):
        # using OceanFlux GHG kt approach with kd based on Wanninkhof2014
        # Wanninkhof, Rik. "Relationship between wind speed and gas exchange over the ocean revisited." Limnology and Oceanography: Methods 12.6 (2014): 351-362.
        function = "(rate_parameterisation.py: k_Wanninkhof2014.__call__)";
        print "%s Using the Wanninkhof 2014 k parameterisation" % (function);
        
        try:
            #for ease of access, simply assign attributes to each input/output.
            for name in self.input_names() + self.output_names():
                setattr(self, name, data[name].fdata);
            data["k"].standardName="W14" 
            data["k"].longName="Wanninkhof 2014, Limnol. Oceanogr.: Methods 12, 2014, 351-362"#IGA
        except KeyError as e:
           print "%s: Required data layer for selected k parameterisation was not found." % function;
           print type(e), e.args;
           return False;
        
        #determine the k relationship
        for i in arange(len(self.k)):   
            self.k[i] = DataLayer.missing_value
            #if ( (self.windu10[i] != DataLayer.missing_value) and (self.windu10_moment2[i] != DataLayer.missing_value) and (self.windu10_moment3[i] != DataLayer.missing_value) and (self.scskin[i] != DataLayer.missing_value) and (self.scskin[i] > 0.0) ):
            if ( (self.windu10[i] != DataLayer.missing_value) and (self.windu10_moment2[i] != DataLayer.missing_value) and (self.scskin[i] != DataLayer.missing_value) and (self.scskin[i] > 0.0) ):#SOCATv4 - no need for wind moment 3
               self.k[i] = 0.251 * self.windu10_moment2[i]
               self.k[i] = self.k[i] * sqrt(660.0/self.scskin[i])
               
               #self.k[i] = self.k[i]# /36.0 # conversion from cm/h to 10^-4 m/s (100/3600) = 1/36
            else:
               self.k[i] = DataLayer.missing_value
        return True;
    
    
    
